# Descrizione lessico ELIta
Il lessico comprende 6905 elementi, tra parole ed emoji. 

Il set di dati contiene il: 
* 21% di aggettivi, 
* il 50% di nomi, 
* il 21% di verbi,
* l'8% di parole che possono essere considerate sia aggettivi che nomi. 

Inoltre, è stato incluso un numero minore di avverbi, espressioni (ad esempio, "restare a bocca aperta" significa "guardare a bocca aperta") e interiezioni (ad esempio, "beh", "boh"). 

La raccolta dei dati ha comportato un processo di annotazione che includeva sia l'associazione delle parole alle emozioni 8 di base (Plutchik), utilizzando una scala da: 
* "non associato" (0), 
* "debolmente associato" (0,25), 
* "moderatamente associato" (0,75),
* "fortemente associato" (1), 

sia la valutazione degli elementi in base alle 3 dimensioni VAD (valenza, eccitazione e dominanza) utilizzando una scala da 1 a 9. 

Ognuna delle 6905 parole/emoji presenti nel lessico ha da un minimo di 5 a un massimo di 10 annotazioni (in media 5,13 annotazioni per parola).

Il lessico è fornito in diverse versioni:
* **RAW**, contiene **tutte** le annotazioni originali fatte dai partecipanti, senza filtri o aggregazioni.
* **GOLDEN**, versione **pulita** della RAW. Sono state selezionate:
    * le 5 annotazioni più simili tra loro per parola,
    * più 1 riga generata automaticamente, "Golden Standard", calcolata dalla moda delle 5 annotazioni migliori.
* **INTENSITY**, versione **aggregata continua**.\
Ogni parola ha una record per emozione (sono state aggiunte: **amore**, media tra gioia e fiducia, e **neutrale**) e dimensione VAD (13 in totale), con la media delle annotazioni selezionate nella versione GOLDEN.
* **BINARY**, versione **aggregata discreta**.\
Converte le annotazioni continue della versione INTENSITY in valori binari (0 o 1) utilizzando una soglia di 0,5.

# Scelta della versione del lessico per la visualizzazione
Per l'obbiettivo di visualizzazione del lessico ELIta con metodi distribuzionali, è stata scelta la versione **INTENSITY** del lessico. 

La ragione, per questa decisione, è che perché INTENSITY fornisce punteggi continui aggregati, necessari per calcolare le distanze semantiche precise tra le parole nello spazio vettoriale PCA/SVD.

Il file è fornito in modo che ci sia una riga per ogni coppia parola-emozione, contenente la media delle annotazioni.\
  Per l'analisi PCA, questo formato verrà trasformato in una matrice vettoriale dove ogni parola ha un unico vettore di punteggi per le 8 emozioni di base (Gioia, Tristezza, Rabbia, Disgusto, Paura, Fiducia, Sorpresa, Anticipazione) più le dimensioni VAD (Valenza, Attivazione, Dominanza) e le categorie aggiuntive (Amore, Neutrale).

# Caricamento e preparazione dati per l'utilizzo
Importiamo il file ELIta_INTENSITY.txt come un DataFrame di pandas. \
Dato che il file non presenta le intestazioni delle colonne, le definiamo qui manualmente. Ciò faciliterà i passaggi successivi.

In [8]:
import pandas as pd

intensityData = pd.read_csv("ELIta_INTENSITY.txt", sep=',', header=None, names=['parola', 'emozione', 'valore'])
intensityData.head()

,parola,emozione,valore
0,‼,gioia,0.71
1,‼,tristezza,0.12
2,‼,rabbia,0.67
3,‼,disgusto,0.04
4,‼,paura,0.17


## Esplorazione del DataFrame
Controllo valori unici nelle colonne 'parola' ed 'emozione' per confermare che il dataset sia stato caricato correttamente.

In [16]:
unique_words = intensityData['parola'].nunique()
unique_emotions = intensityData['emozione'].nunique()
print(f"Numero di parole uniche: {unique_words}")
print(f"Numero di emozioni/dimensioni uniche: {unique_emotions}")

Numero di parole uniche: 6905
Numero di emozioni/dimensioni uniche: 13


Controllo per valori mancanti nel DataFrame.

In [17]:
total_nulls = intensityData.isnull().sum().sum()
print(f"Totale valori mancanti nel dataset: {total_nulls}")

Totale valori mancanti nel dataset: 0


## Trasformazione del DataFrame in una matrice vettoriale
Manipolazione del dataset in una matrice per avere una riga per ogni parola e una colonna per ogni emozione/dimensione.

In [18]:
pivotedData = intensityData.pivot(index='parola', columns='emozione', values='valore')
pivotedData.head()

emozione,amore,aspettativa,attivazione,disgusto,dominanza,fiducia,gioia,neutrale,paura,rabbia,sorpresa,tristezza,valenza
parola,,,,,,,,,,,,,
????,0.08,0.17,7.00,0.17,4.33,0.04,0.12,0.0,0.17,0.42,0.79,0.42,5.17
a_caso,0.04,0.21,5.00,0.08,3.00,0.04,0.04,0.0,0.54,0.29,0.83,0.21,4.33
a_malincuore,0.00,0.12,5.50,0.21,3.67,0.00,0.00,0.0,0.21,0.21,0.04,0.83,3.33
a_scanso_di,0.21,0.58,5.33,0.04,4.67,0.25,0.17,0.0,0.17,0.42,0.21,0.25,5.00
abbagliante,0.32,0.62,7.00,0.00,6.50,0.17,0.46,0.0,0.12,0.12,0.71,0.04,6.67


## Salvataggio della matrice vettoriale
Salviamo la matrice vettoriale in un file CSV per utilizzi futuri.

In [21]:
pivotedData.to_csv("ELIta_INTENSITY_Matrix.csv")

---